# Daily Digest

In [7]:
import datetime
import docx
import pandas as pd
import oracledb
import smtplib
import subprocess
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Local save dir (create if not exists)
save_dir = os.path.expanduser('~/equilend_daily')
os.makedirs(save_dir, exist_ok=True)

# Clouds to sync to (tweak remotes as per your rclone config, e.g., 'google_drive:daily_digest/', 'onedrive:equilend/')
cloud_remotes = [
    'google_drive:daily_digest/',  # Example: Google Drive folder
    'onedrive:equilend_daily/',     # Example: OneDrive folder
    # Add more like 'dropbox:backups/', 'icloud:docs/'
]

# Load the Word doc (assume filename: 'daily_content.docx')
doc = docx.Document('daily_content.docx')

# Extract section function (robust for multi-line)
def extract_section(doc, start_header, end_header=None):
    content = []
    capturing = False
    for para in doc.paragraphs:
        text = para.text.strip()
        if text == start_header:
            capturing = True
            continue
        if end_header and text == end_header:
            capturing = False
            break
        if capturing and text:
            content.append(text)
    return content

# Market Notes
market_notes = extract_section(doc, '📈 Market Notes', '🔥 Major Headlines & What Our Data Shows')

# Headlines: handle multi-line
headlines = []
title = ''
desc = ''
data_shows = ''
in_desc = False
headlines_section = extract_section(doc, '🔥 Major Headlines & What Our Data Shows', '📊 Today\'s Specials & Hard-to-Borrow')
for para in headlines_section:
    if para.isupper() and not para.startswith('What Our Data Shows:'):
        if title:
            headlines.append({'title': title, 'description': desc.strip(), 'data_shows': data_shows.replace('What Our Data Shows: ', '')})
        title = para
        desc = ''
        data_shows = ''
        in_desc = True
    elif para.startswith('What Our Data Shows: '):
        data_shows = para
        in_desc = False
    elif in_desc:
        desc += ' ' + para
# Append last
if title:
    headlines.append({'title': title, 'description': desc.strip(), 'data_shows': data_shows.replace('What Our Data Shows: ', '')})

# Key Takeaways
key_takeaways = extract_section(doc, '💡 Key Takeaways', 'Source: EquiLend Data & Analytics')

# Oracle DB connection
dsn = 'DATALENDDR'
user = 'EQMRKS_ANA'
password = 'rEN8mmNrMP'
conn = oracledb.connect(user=user, password=password, dsn=dsn)

# Dynamic date range: last 30 days for history to compute diffs
today = datetime.date.today()
from_date = (today - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
to_date = (today - datetime.timedelta(days=1)).strftime('%Y-%m-%d')  # Yesterday for latest

# Corrected SQL to pull raw data (no diffs; compute in Python)
sql = f"""
SELECT 
    r.business_date,
    s.ticker,
    s.security_description AS name,
    a.shares * a.security_price_c AS market_cap,
    a.shares AS sharesoutstanding,
    c.country_name,
    i.sector_name,
    i.industry_name AS industry,
    d.return_loan_qty_amt,
    d.new_loan_qty_amt,
    a.security_price_c AS price,
    d.loan_qty_amt_c,
    a.short_squeeze_score_c AS short_squeeze_score,
    d.avg_spread_all_amt_c AS fee_bps,
    a.short_interest_c AS short_interest,
    a.days_to_cover_c AS days_to_cover,
    d.active_utilization_c AS active_utilization,
    d.loan_val_amt_c AS on_loan_value,
    d.bb_val_amt_c,
    d.bb_qty_amt_c AS borrower_count,  # Assuming this as proxy for borrower count; update field if different
    d.active_lendable_qty_c,
    d.active_lendable_val_c
FROM 
    (SELECT MAX(result_id) result_id, business_date 
     FROM md_result 
     WHERE business_date BETWEEN '{from_date}' AND '{to_date}' 
     GROUP BY business_date) r
INNER JOIN md_result_details d ON d.result_id = r.result_id
INNER JOIN md_security s ON s.md_security_id = d.md_security_id
INNER JOIN md_sec_addin a ON a.md_security_id = s.md_security_id AND a.result_id = r.result_id
LEFT JOIN idc_sec_ind_map i ON i.sub_industry_id = s.industry_cd
INNER JOIN country c ON c.country_cd = s.country_cd
"""

# Pull data and save raw CSV
df = pd.read_sql_query(sql, conn)
conn.close()
yesterday = today - datetime.timedelta(days=1)
csv_path = os.path.join(save_dir, f"raw_db_pull_{yesterday.strftime('%Y-%m-%d')}.csv")
df.to_csv(csv_path, index=False)

# Process data: convert date, sort, compute diffs (absolute and % where relevant; WoW ~5 business days)
df['business_date'] = pd.to_datetime(df['business_date'])
df = df.sort_values(['ticker', 'business_date'])

# Compute WoW changes (shift 5 for approx week, assuming business days)
df['fee_wow'] = df.groupby('ticker')['fee_bps'].diff(periods=5)
df['price_wow_pct'] = df.groupby('ticker')['price'].pct_change(periods=5) * 100
df['short_squeeze_score_wow'] = df.groupby('ticker')['short_squeeze_score'].diff(periods=5)

# Latest data only
latest_date = df['business_date'].max()
latest_df = df[df['business_date'] == latest_date].copy()

# For specials table: filter positive WoW momentum, sort by fee desc, top 5
latest_df['fee_num'] = latest_df['fee_bps'].apply(lambda x: parse_fee(str(x)))  # Reuse parse_fee from before
latest_df['mom_num'] = latest_df['fee_wow'].apply(lambda x: parse_momentum(str(x)))
filtered_specials = latest_df[latest_df['mom_num'] > 0].sort_values(by='fee_num', ascending=False).head(5)
filtered_specials['Fee (BPS)'] = filtered_specials['fee_bps'].apply(band_fee)
filtered_specials['Momentum (WoW)'] = filtered_specials['fee_wow'].apply(lambda x: f'+{x:,.0f} BPS' if pd.notnull(x) else '')  # Format as +N BPS
specials_cols = ['ticker', 'name', 'industry', 'Fee (BPS)', 'Momentum (WoW)']
filtered_specials = filtered_specials[specials_cols].rename(columns={'ticker': 'Ticker', 'name': 'Name', 'industry': 'Industry'})

# For squeeze table: filter score >60, on_loan >1M, borrower >=2, sort by score desc, top 5
filtered_squeeze = latest_df[(latest_df['short_squeeze_score'] > 60) &
                             (latest_df['on_loan_value'] > 1000000) &
                             (latest_df['borrower_count'] >= 2)].sort_values(by='short_squeeze_score', ascending=False).head(5)
filtered_squeeze['Price (WoW)'] = filtered_squeeze['price_wow_pct'].apply(lambda x: f'{x:.1f}%' if pd.notnull(x) else '')
filtered_squeeze['Score (WoW)'] = filtered_squeeze['short_squeeze_score_wow'].apply(lambda x: f'{x:+.2f}' if pd.notnull(x) else '')
squeeze_cols = ['ticker', 'name', 'industry', 'price', 'Price (WoW)', 'short_squeeze_score', 'Score (WoW)']
filtered_squeeze = filtered_squeeze[squeeze_cols].rename(columns={'ticker': 'Ticker', 'name': 'Company Name', 'industry': 'Industry', 'price': 'Price', 'short_squeeze_score': 'Short Squeeze Score'})

# Fee banding and parse functions (from before)
def band_fee(fee):
    try:
        fee_num = float(str(fee).replace(',', '').replace(' BPS', '').split(' to ')[-1] if ' to ' in str(fee) else str(fee).replace('+', '').replace(' BPS', ''))
    except ValueError:
        return fee
    if fee_num >= 50000:
        return "50,000+"
    elif fee_num >= 30000:
        return "30,000 to 49,999"
    elif fee_num >= 10000:
        return "10,000 to 29,999"
    elif fee_num >= 5000:
        return "5,000 to 9,999"
    elif fee_num >= 1000:
        return "1,000 to 4,999"
    else:
        return f"{int(fee_num)}"

def parse_fee(fee_str):
    fee_str = str(fee_str).replace(',', '').replace(' BPS', '')
    if '+' in fee_str:
        return float(fee_str.replace('+', ''))
    elif ' to ' in fee_str:
        _, high = map(float, fee_str.split(' to '))
        return high
    else:
        try:
            return float(fee_str)
        except ValueError:
            return 0.0

def parse_momentum(mom_str):
    mom_str = str(mom_str).replace(',', '').replace(' BPS', '').replace('+', '').lstrip('-')
    try:
        return float(mom_str)
    except ValueError:
        return 0.0

# Yesterday's date
yesterday = today - datetime.timedelta(days=1)
date_str = yesterday.strftime('%B %d, %Y')

# HTML with branding
html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>EquiLend D&A Daily Digest</title>
    <style>
        body {{ font-family: Arial, sans-serif; color: #333; background-color: #f4f4f4; margin: 20px; }}
        h1 {{ color: #003366; text-align: center; }}
        h2 {{ color: #003366; }}
        .section {{ margin-bottom: 20px; }}
        ul {{ list-style-type: disc; margin-left: 20px; }}
        table {{ width: 100%; border-collapse: collapse; margin-top: 10px; }}
        th, td {{ border: 1px solid #808080; padding: 8px; text-align: left; }}
        th {{ background-color: #003366; color: white; }}
        .source {{ text-align: right; font-style: italic; color: #808080; }}
    </style>
</head>
<body>
    <h1>EquiLend D&A Daily Digest</h1>
    <p style="text-align: center;">{date_str}</p>

    <div class="section">
        <h2>📈 Market Notes</h2>
        <ul>
            <li>{market_notes[0] if market_notes else ''}</li>
            <li>{market_notes[1] if len(market_notes) > 1 else ''}</li>
        </ul>
    </div>

    <div class="section">
        <h2>🔥 Major Headlines & What Our Data Shows</h2>
        {"".join([f"""
        <p><strong>{hl['title']}</strong></p>
        <p>{hl['description']}</p>
        <p><strong>What Our Data Shows:</strong> {hl['data_shows']}</p>
        """ for hl in headlines])}
    </div>

    <div class="section">
        <h2>📊 Today's Specials & Hard-to-Borrow</h2>
        <table>
            <thead>
                <tr>
                    <th>Ticker</th>
                    <th>Name</th>
                    <th>Industry</th>
                    <th>Fee (BPS)</th>
                    <th>Momentum (WoW)</th>
                </tr>
            </thead>
            <tbody>
                {''.join([f"<tr><td>{row['Ticker']}</td><td>{row['Name']}</td><td>{row['Industry']}</td><td>{row['Fee (BPS)']}</td><td>{row['Momentum (WoW)']}</td></tr>" for _, row in filtered_specials.iterrows()])}
            </tbody>
        </table>

        <br>

        <table>
            <thead>
                <tr>
                    <th>Ticker</th>
                    <th>Company Name</th>
                    <th>Industry</th>
                    <th>Price</th>
                    <th>Price (WoW)</th>
                    <th>Short Squeeze Score</th>
                    <th>Score (WoW)</th>
                </tr>
            </thead>
            <tbody>
                {''.join([f"<tr><td>{row['Ticker']}</td><td>{row['Company Name']}</td><td>{row['Industry']}</td><td>${row['Price']:.2f}</td><td>{row['Price (WoW)']}</td><td>{row['Short Squeeze Score']:.2f}</td><td>{row['Score (WoW)']}</td></tr>" for _, row in filtered_squeeze.iterrows()])}
            </tbody>
        </table>
    </div>

    <div class="section">
        <h2>💡 Key Takeaways</h2>
        <ul>
            {"".join([f"<li>{kt}</li>" for kt in key_takeaways])}
        </ul>
    </div>

    <p class="source">Source: EquiLend Data & Analytics</p>
</body>
</html>
"""

# Write HTML file
html_path = os.path.join(save_dir, f"equilend_daily_digest_{yesterday.strftime('%Y-%m-%d')}.html")
with open(html_path, 'w') as f:
    f.write(html_template)

# Sync to clouds
files_to_sync = [csv_path, html_path]
for file_path in files_to_sync:
    for remote in cloud_remotes:
        try:
            subprocess.call(['rclone', 'copy', file_path, remote])
            print(f"Synced {os.path.basename(file_path)} to {remote}")
        except Exception as e:
            print(f"Error syncing {os.path.basename(file_path)} to {remote}: {e}")

# Generate email draft
sender = 'rsheehan@equilend.com'
password = 'Nice2389..'  # Fake for now
recipient = 'bob.sheehan@equilend.com'

msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = recipient
msg['Subject'] = f"Draft: EquiLend D&A Daily Digest - {date_str}"
msg.attach(MIMEText(html_template, 'html'))

try:
    server = smtplib.SMTP('smtp.office365.com', 587)
    server.starttls()
    server.login(sender, password)
    server.send_message(msg)
    server.quit()
    print("Email draft sent successfully!")
except Exception as e:
    print(f"Error sending email: {e}"

SyntaxError: incomplete input (3967947773.py, line 323)